## -1 = Phishing, 0 = Suspicious, 1 = Legitimate

In [1]:
import pickle
filename = 'PhishingSites_model1.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [2]:
from IPy import IP
import numpy as np
import whois
import re
import pandas as pd
from urllib.request import Request, urlopen, ssl, socket
from urllib.error import URLError, HTTPError
import json
from urllib.parse import urlparse
import ast
import whois
import favicon
import requests
import urllib, bs4

data = pd.read_excel('ccTLD.xlsx')
urlShorteningServices = ['bit.ly','goo.gl','t.co','tinyurl','youtu.be','ow.ly']
ccTLD = data.iloc[:,0].values
CA = ["comodo","identrust","symantec","godaddy","globalsign","digicert","certum","entrust","secom",
      "actalis","trustwave","let's encrypt","startcom","wisekey group","verizon","unizeto","buypass",
     "quovadis","deutsche telekom","network solutions","swisssign","google","name.com","1&1 ionos"
     "123-reg","arq group","cozareg","domainz","dotconnectafrica","dreamhost","drop registrar",
     "enom","gandi","hostdime","hover","internet initiative japan","japan registry ervices",
     "namecheap","net4","rebel.com","register.com","samoanic","uniregistry","registrarsafe","markmonitor"]

In [3]:
parameters = []

def having_IP_Address(url):
    link = str(url)
    link = link.split('/')
    try:
        x = re.match("^(([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])\.){3}([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])$",link[2])
        if x is None:
            value = np.int(1)
        else:
            value = np.int(-1)
    except:
        value = np.int(0)
    parameters.append(value)

def URL_Length(url):
    try:
        length = len(url)
        if length < 54:
            value = np.int(1)
        elif length >= 54 & length <= 75:
            value = np.int(0)
        else:
            value = np.int(-1)
    except:
        value = np.int(-1)
    parameters.append(value)

def Shortining_Service(url):
    try:
        for item in urlShorteningServices:
            if item in url:
                value = np.int(-1)
            else:
                value = np.int(1)
    except:
        value = np.int(0)
    parameters.append(value)

def having_At_Symbol(url):
    try:
        if '@' in url:
            value = np.int(-1)
        else:
            value = np.int(1)
    except:
        value = np.int(1)
    parameters.append(value)

def double_slash_redirecting(url):
    try:
        location = url[8::1].index('//')
        if location > 7:
            value = np.int(-1)
    except:
        value = np.int(1)
    parameters.append(value)

def Prefix_Suffix(url):
    try:
        url = url[8:-3:1]
        if '-' in url:
            value = np.int(-1)
        else:
            value = np.int(1)
    except:
        value = np.int(0)
    parameters.append(value)

def having_Sub_Domain(url):
    try:
        url = url.replace('www.','')
        for item in ccTLD:
            if item in url:
                url = url.replace(item,'')
        dots = url.count('.')
        if dots <= 1:
            value = np.int(1)
        elif dots == 2:
            value = np.int(0)
        else:
            value = np.int(-1)
    except:
        value = np.int(-1)
    parameters.append(value)

def HTTPS_token(url):
    try:
        location = url[5::1].index('http')
        if location >= 0:
            value = np.int(-1)
    except:
        value = np.int(1)
    parameters.append(value)

def SSLfinal_State(url):
    try:
        values = []
        parsed_uri = urlparse(url)
        scheme = str(parsed_uri.scheme)

        w = whois.whois(url)
        issuer = w["registrar"].lower()

        created = str(w["creation_date"])
        expires = str(w["expiration_date"])
        created_year = int(created[:4])
        expires_year = int(expires[:4])
        age = int(expires_year - created_year)

        for name in CA:
            index = issuer.find(name)
            values.append(index)
        trustedIssuer = max(values)
        if scheme == 'https' and trustedIssuer >= 0 and age >= 1:
            value = np.int(1)
        elif scheme == 'https' and trustedIssuer == -1:
            value = np.int(0)
        else:
            value = np.int(-1)
    except:
        value = np.int(-1)
    parameters.append(value)

def Domain_registeration_length(url):
    try:
        w = whois.whois(url)
        updated = str(w["updated_date"])
        expires = str(w["expiration_date"])
        updated_year = int(updated[:4])
        expires_year = int(expires[:4])
        age = int(expires_year - updated_year)
        if age <= 1:
            value = np.int(-1)
        else:
            value = np.int(1)
    except:
        value = np.int(-1)
    parameters.append(value)

def Favicon(url):
    try:
        icons = favicon.get(url)
        parsed_uri = urlparse(url)
        base_url = '{uri.netloc}'.format(uri=parsed_uri)
        values = []
        for item in icons:
            item = str(item).split("url=",1)[1]
            item = str(item).split(", width",1)[0]
            if str(base_url) in str(item):
                values.append(item)
        if len(values) > 0:
            value = np.int(1)
        else:
            value = np.int(0)
    except:
        value = np.int(-1)
    parameters.append(value)

def Redirect(url):
    try:
        values = []
        def chase_redirects(url):
            while True:
                yield url
                r = requests.head(url)
                if 300 < r.status_code < 400:
                    url = r.headers['location']
                else:
                    break

        for link in chase_redirects(url):
            values.append(link)

        if len(values) <= 1:
            value = np.int(1)
        elif len(values) >= 2 and len(values) < 4:
            value = np.int(0)
        else:
            value = np.int(-1)
    except:
        value = np.int(-1)
    parameters.append(value)

def age_of_domain(url):
    try:
        w = whois.whois(url)
        days = str(w['expiration_date'] - w['creation_date'])
        age = int(days.split(" days",1)[0])
        if age >= 183:
            value = np.int(1)
    except:
        value = np.int(-1)
    parameters.append(value)

def DNSRecord(url):
    try:
        w = whois.whois(url)
        value = np.int(1)
    except:
        value = np.int(-1)
    parameters.append(value)

def web_traffic(url):
    try:
        rank = int(bs4.BeautifulSoup(urlopen("http://data.alexa.com/data?cli=10&dat=s&url="+ url).read(),"xml").find("REACH")['RANK'])
        if rank < 100000:
            value = np.int(1)
        elif rank > 100000:
            value = np.int(0)
    except:
        value = np.int(-1)
    parameters.append(value)

def Abnormal_URL(url):
    try:
        w = whois.whois(url)
        name = w["domain_name"].lower()
        if name in str(url):
            value = np.int(1)
    except:
        value = np.int(-1)
    parameters.append(value)

In [4]:
url = "http://https-www-paypal-it-webapps-mpp-home.soft-hair.com/"
#url = "http://yushuishuo.com/wp-content/amex/americanexpress.com-login.verify-authenticate/home/index2.php"
parameters = []

having_IP_Address(url)
URL_Length(url)
Shortining_Service(url)
having_At_Symbol(url)
double_slash_redirecting(url)
Prefix_Suffix(url)
having_Sub_Domain(url)
SSLfinal_State(url)
Domain_registeration_length(url)
Favicon(url)
HTTPS_token(url)
Abnormal_URL(url)
Redirect(url)
age_of_domain(url)
DNSRecord(url)
web_traffic(url)

mode = max(set(parameters), key=parameters.count)

Page_Rank = np.int(mode)
parameters.append(Page_Rank)
Google_Index = np.negative(mode)
parameters.append(Google_Index)
Links_pointing_to_page = np.int(mode)
parameters.append(Links_pointing_to_page)
Statistical_report = np.int(mode)
parameters.append(Statistical_report)

#parameters

In [6]:
newParameters = pd.DataFrame({"having_IP_Address":[parameters[0]], "URL_Length":[parameters[1]], "Shortining_Service":[parameters[2]],
                             "having_At_Symbol":[parameters[3]], "double_slash_redirecting":[parameters[4]], "Prefix_Suffix":[parameters[5]],
                             "having_Sub_Domain":[parameters[6]], "SSLfinal_State":[parameters[7]], "Domain_registeration_length":[parameters[8]],
                             "Favicon":[parameters[9]], "HTTPS_token":[parameters[10]], "Abnormal_URL":[parameters[11]], "Redirect":[parameters[12]],
                             "age_of_domain":[parameters[13]], "DNSRecord":[parameters[14]], "web_traffic":[parameters[15]], "Page_Rank":[parameters[16]],
                             "Google_Index":[parameters[17]], "Links_pointing_to_page":[parameters[18]], "Statistical_report":[parameters[19]]})

In [7]:
newParameters

,Abnormal_URL,DNSRecord,Domain_registeration_length,Favicon,Google_Index,HTTPS_token,Links_pointing_to_page,Page_Rank,Prefix_Suffix,Redirect,SSLfinal_State,Shortining_Service,Statistical_report,URL_Length,age_of_domain,double_slash_redirecting,having_At_Symbol,having_IP_Address,having_Sub_Domain,web_traffic
0,1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,-1,0,-1,1,1,1,1,-1


In [8]:
prediction = loaded_model.predict(newParameters)
int(prediction)

-1

In [13]:
loaded_model.predict_proba(newParameters)

array([[ 0.82,  0.18]])